# Treinamento

## Inicialização

In [5]:
# Config Inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.metrics import *

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

### MLFlow

In [6]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("ecd15")

<Experiment: artifact_location='file:///C:/Estudo/ECD15-MLOps/models/mlruns/0', creation_time=1743297663367, experiment_id='0', last_update_time=1743297701488, lifecycle_stage='active', name='ecd15', tags={}>

In [7]:
seeds = [42, 47, 123]

## Dataset

O pré-processamento inicial foi realizado no dataset `brasile-real-state-dataset.csv`, onde as colunas de latitude e longitude foram convertidas em informações de cidade e estado. Esse processo resultou na criação do dataset `brasil_estado_cidade.csv`, que está sendo utilizado para o treinamento dos modelos apresentados a seguir.


### Leitura

In [8]:
# Carregar o conjunto de dados
dados = pd.read_csv("dataset/brasil_estado_cidade.csv", encoding="utf-8")

# Eliminando registros com valores null
dados.dropna(inplace=True)

# float64
dados = dados.astype({col: 'float64' for col in dados.select_dtypes(include='int').columns})

def remover_outliers_por_cidade(df):
    # Remove outliers da coluna 'price_brl' agrupando por cidade.

    def remover_outliers_grupo(grupo):
        # Remove outliers de um grupo usando o método Z-score.
        z_scores = np.abs(stats.zscore(grupo["price_brl"]))
        return grupo[(z_scores < 3)]

    df_filtrado = df.groupby("city").apply(remover_outliers_grupo).reset_index(drop=True)
    return df_filtrado

dados = remover_outliers_por_cidade(dados)

dados.sample(10)

/tmp/ipykernel_148/3995125492.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df.groupby("city").apply(remover_outliers_grupo).reset_index(drop=True)


,Unnamed: 0,property_type,state,region,lat,lon,area_m2,price_brl,city
8974,5809.0,apartment,Rio de Janeiro,Southeast,-22.872278,-43.028980,53.0,329382.13,Sete Pontes
717,8352.0,apartment,Santa Catarina,South,-26.985996,-48.636383,76.0,499063.83,Balneário Camboriú
8108,6339.0,apartment,Rio de Janeiro,Southeast,-22.914837,-43.182472,75.0,538988.95,Rio de Janeiro
5606,3331.0,apartment,Rio Grande do Sul,South,-30.015261,-51.165073,87.0,548970.21,Porto Alegre
4257,1259.0,apartment,Rio Grande do Sul,South,-30.031038,-51.217381,90.0,666332.92,Porto Alegre
2879,230.0,apartment,Pernambuco,Northeast,-8.215346,-34.927231,103.0,469120.01,Jaboatão dos Guararapes
8852,12225.0,house,So Paulo,Southeast,-23.648643,-46.543930,127.0,548970.21,Santo André
9585,10306.0,house,So Paulo,Southeast,-23.501787,-46.681152,200.0,479101.27,São Paulo
8375,1768.0,apartment,Rio Grande do Sul,South,-29.700670,-52.443275,72.0,334572.82,Santa Cruz do Sul
7294,5154.0,apartment,Rio de Janeiro,Southeast,-22.899942,-43.281296,73.0,409232.33,Rio de Janeiro


### Tratamento Features

In [9]:
# Exemplo de pré-processamento
x_features = dados.drop(["price_brl"], axis=1)  # Features
y_target = dados["price_brl"]  # Variável alvo

# Identificando colunas numéricas e categóricas
numeric_features = x_features.select_dtypes(include=['number']).columns
categorical_features = x_features.select_dtypes(include=['object']).columns

# Criando transformadores
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Criando o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


### Divisão dos Dados

In [10]:
test_sizes = [0.1, 0.2, 0.3]
#x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=0.2, random_state=42)

## Modelos

- Árvore de Decisão: `DecisionTreeRegressor`
- Regressão Linear: `LinearRegression`
- Random Forest: `RandomForestRegressor`
- Gradient Boosting: `GradientBoostingRegressor`
- XGBoost: `XGBRegressor`

In [11]:
models = {
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42)
}

## Execução

In [12]:
# para cada semente 
for seed in seeds:
    # para cada tamanho de teste
    for test_size in test_sizes:
        # Dividir os dados em conjuntos de treino e teste
        x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=test_size, random_state=seed)

        # Para cada modelo
        for model_name, model in models.items():
            # Criar o pipeline
            pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('model', model)])

            # Treinar o modelo
            pipeline.fit(x_train, y_train)

            # Fazer previsões
            y_pred = pipeline.predict(x_test)

            # Avaliar o modelo
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            
            print(f"Modelo: {model_name}, Test Size: {test_size}, Seed: {seed}")
            print(f"MSE: {mse}, R2: {r2}, MAE: {mae}")
            
            # Logar os resultados no MLflow
            with mlflow.start_run():
                mlflow.log_param("model_type", model_name)
                mlflow.log_param("test_size", test_size)
                mlflow.log_param("random_seed", seed)
                
                # registrar métricas
                mlflow.log_metric("mse", mse)
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("mae", mae)

                # Logar o modelo
                signature = infer_signature(x_test, y_pred)
                
                registered_model_name= model_name.lower().replace(" ", "_") + "_model"
                 
                mlflow.sklearn.log_model(
                    sk_model=pipeline,
                    artifact_path="'sklearn'-model",
                    signature=signature,
                    registered_model_name= registered_model_name,
                )
                
            mlflow.end_run()
                

Modelo: Decision Tree, Test Size: 0.1, Seed: 42
MSE: 61318563319.787544, R2: 0.4643648515387715, MAE: 173409.66728007182


Registered model 'decision_tree_model' already exists. Creating a new version of this model...
Created version '23' of model 'decision_tree_model'.


Modelo: Linear Regression, Test Size: 0.1, Seed: 42
MSE: 53575827203.35227, R2: 0.5319998609827157, MAE: 173412.6447177572


Registered model 'linear_regression_model' already exists. Creating a new version of this model...
Created version '23' of model 'linear_regression_model'.


Modelo: Random Forest, Test Size: 0.1, Seed: 42
MSE: 35609940151.11541, R2: 0.6889370111288469, MAE: 132206.2374586176


...

Registered model 'random_forest_model' already exists. Creating a new version of this model...
Created version '30' of model 'random_forest_model'.


Modelo: Gradient Boosting, Test Size: 0.3, Seed: 123
MSE: 39765471976.7416, R2: 0.6523843262185738, MAE: 148802.22336159798


Registered model 'gradient_boosting_model' already exists. Creating a new version of this model...
Created version '30' of model 'gradient_boosting_model'.


Modelo: XGBoost, Test Size: 0.3, Seed: 123
MSE: 35728929338.27869, R2: 0.6876703524937664, MAE: 137394.49164296337


Registered model 'xgboost_model' already exists. Creating a new version of this model...
Created version '30' of model 'xgboost_model'.


In [ ]:

#Avaliar modelos, corrigir valores de acordo com a inflação,  testar drift, 

#df_examples, y = load_new_data()
#drift_score, drift_by_columns = evaluate_model(df_examples, y, None)
#new_data = simulate_drift(df_examples)
#drift_score, drift_by_columns = evaluate_model(df_examples, y, new_data)
#check_for_drift(drift_score, drift_by_columns)

